In [ ]:
import os
import pandas as pd
import json
from svm import *
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
ABOSOLUTE_PATH = "/"
SVM_PREDICTION = ABOSOLUTE_PATH + 'SVM/Prediction/'
PATH_FOLDS_BOW_AND_NGRAM =  ABOSOLUTE_PATH + 'SVM/BOWNgram/5folds/'
PATH_FOLDS_BOW = ABOSOLUTE_PATH + 'SVM/BOW/5folds/'

In [ ]:
def predict(fileObject, baseToPredict, feature, rootTarget):
    if not os.path.exists(rootTarget):
        os.makedirs(rootTarget)
    log = {}
        
    svmClassifier = readObjectInFile(fileObject)
    prediction = svmClassifier.predict(feature)    
    savePrediction(baseToPredict, prediction, rootTarget)
    
    log["classifier"] = svmClassifier.get_params(True)
    report = classification_report(baseToPredict['rotulo'], prediction, output_dict=True)
    log["report"] = report

    confusionMatrix = confusion_matrix(baseToPredict['rotulo'], prediction)
    log["confusionMatrix"] = str(confusionMatrix)
    with open(rootTarget + PREDICTION_LOG, 'w') as filelog:
        filelog.write(json.dumps(log, indent=1))
        filelog.close()

In [ ]:
def GeneratePrediction(base, dirToThisPrediction, BOWAndNgram):
    if not os.path.exists(SVM_PREDICTION):
        os.makedirs(SVM_PREDICTION)
    root = PATH_FOLDS_BOW_AND_NGRAM if BOWAndNgram else PATH_FOLDS_BOW
    rootTarget = SVM_PREDICTION + ('BowAndNgram/' if BOWAndNgram else 'Bow/')
    vectorizer = readObjectInFile( root + VECTORIZER_OBJECT_FILE)
    feature = getFeatures(vectorizer, base)
    
    for dirname in os.listdir(PATH_FOLDS_BOW_AND_NGRAM if BOWAndNgram else PATH_FOLDS_BOW):
        if dirname != VECTORIZER_OBJECT_FILE:
            print("fold: " + dirname)    
            predict(root + dirname + "/" + PYTHON_OBJECT_FILE,
                    base,
                    feature,
                    rootTarget + dirname + "/" + dirToThisPrediction)

**Example execution**

base = getBaseDF([ABOSOLUTE_PATH + 'data/class1.xlsx', ABOSOLUTE_PATH + 'data/class2.xlsx'])

GeneratePrediction(base, 'dirTargetName/', True)